In [1]:
import dask
from dask.distributed import Client, LocalCluster, performance_report

In [2]:
# Does it help? Going to turn this off. 
# Nope does not help.

# dask.config.set({
#     "distributed.comm.compression": "zstd",
#     "distributed.comm.zstd.level": 1
# })


In [2]:
client = Client(LocalCluster(n_workers=16, threads_per_worker=1, processes=True))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 121.12 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39271,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 121.12 GiB
Comm: tcp://127.0.0.1:44321,Total threads: 1
Dashboard: http://127.0.0.1:44213/status,Memory: 7.57 GiB
Nanny: tcp://127.0.0.1:44569,


In [ ]:
"""
Runs fusion from config file generated 
from dispim or exaspim scheduler. 
"""

import glob
import os
import uuid
import time
from pathlib import Path
import yaml

import torch
import numpy as np

import aind_cloud_fusion.blend as blend
import aind_cloud_fusion.fusion as fusion
import aind_cloud_fusion.geometry as geometry
import aind_cloud_fusion.io as io

def read_config_yaml(yaml_path: str) -> dict:
    with open(yaml_path, "r") as f:
        yaml_dict = yaml.safe_load(f)
    return yaml_dict

def run_exaspim_worker(yml_path: str, 
                       xml_path: str,
                       output_path: str): 
    # Parse information from worker yaml
    # (See dispim_scheduler data contract)
    configs = read_config_yaml(yml_path)
    input_path = configs['input_path']
    output_s3_path = configs['output_path']
    worker_cells = [tuple(cell) for cell in configs['worker_cells']] 

    # Application Object: DATASET
    xml_path = str(Path(xml_path))
    s3_path = input_path
    DATASET = io.BigStitcherDataset(xml_path, s3_path)

    # Application Object: OUTPUT_PARAMS
    OUTPUT_PARAMS = io.OutputParameters(
        path=output_s3_path,
        chunksize=(1, 1, 128, 128, 128),
        resolution_zyx=(1.0, 0.748, 0.748),
    )

    # Application Object: RUNTIME PARAMS
    RUNTIME_PARAMS = io.RuntimeParameters(
        use_gpus=False,
        devices=[torch.device("cpu")],
        pool_size=int(os.environ.get("CO_CPUS", 1)),   # Retrieve CO cpu env var 
        worker_cells=worker_cells
    )
    # Application Parameter: CELL_SIZE
    CELL_SIZE = [128, 128, 128]

    # Application Parameter: POST_REG_TFMS
    POST_REG_TFMS: list[geometry.Affine] = []
    
    # Application Object: BLENDING_MODULE
    BLENDING_MODULE = blend.MaxProjection()

    # Run fusion
    fusion.run_fusion(
            DATASET,
            OUTPUT_PARAMS,
            RUNTIME_PARAMS,
            CELL_SIZE,
            POST_REG_TFMS,
            BLENDING_MODULE,
    )

    # Log 'done' file for next capsule in pipeline. 
    # Unique log filename
    unique_id = str(uuid.uuid4())
    timestamp = int(time.time() * 1000)
    unique_file_name = str(Path(output_path) / f"file_{timestamp}_{unique_id}.yml")

    log_content = {}
    log_content['output_path'] = OUTPUT_PARAMS.path
    log_content['resolution_zyx'] = list(OUTPUT_PARAMS.resolution_zyx)

    with open(unique_file_name, "w") as file:
        yaml.dump(log_content, file)

def run_dispim_worker(yml_path: str, 
                      xml_path: str,
                      output_path: str):
    # Parse information from worker yaml
    # (See dispim_scheduler data contract)
    configs = read_config_yaml(yml_path)
    dataset_type = configs['dataset_type']
    if 'channel' in configs: 
        channel = int(configs['channel'])
    input_path = configs['input_path']
    output_s3_path = configs['output_path']

    # Initialize Application Objects
    # Application Object: DATASET
    if dataset_type == 'BigStitcherDataset':
        xml_path = str(Path(xml_path))
        s3_path = input_path
        dataset = io.BigStitcherDataset(xml_path, s3_path)
    elif dataset_type == 'BigStitcherDatasetChannel': 
        xml_path = str(Path(xml_path))
        s3_path = input_path
        channel_num = channel
        dataset = io.BigStitcherDatasetChannel(xml_path, s3_path, channel_num)
    DATASET = dataset

    # Application Object: OUTPUT_PARAMS
    OUTPUT_PARAMS = io.OutputParameters(
        path=output_s3_path,
        chunksize=(1, 1, 128, 128, 128),
        resolution_zyx=(0.5, 0.5, 0.5),
    )

    # Application Object: RUNTIME PARAMS
    # Will define worker_cells at the end
    RUNTIME_PARAMS = io.RuntimeParameters(
        use_gpus=False,
        devices=[torch.device("cpu")],
        pool_size=int(os.environ.get("CO_CPUS", 1))   # Retrieve CO cpu env var
    )

    # Application Parameter: CELL_SIZE
    CELL_SIZE = [128, 128, 128]

    # Application Parameter: POST_REG_TFMS
    POST_REG_TFMS: list[geometry.Affine] = []
    matrix = [[1, 0, -1, 0], 
              [0, 1, 0, 0], 
              [0, 0, 1, 0]]
    deskewing = geometry.Affine(np.array(matrix))
    POST_REG_TFMS.append(deskewing)

    # Application Object: BLENDING_MODULE
    BLENDING_MODULE = blend.MaxProjection()

    # Finally, initalize and run fusion. 
    # For the dispim pipeline, each worker will run on an entire channel. 
    worker_cells = []
    _, _, _, _, output_volume_size, _ = fusion.initialize_fusion(
        DATASET, POST_REG_TFMS, OUTPUT_PARAMS
    )
    z_cnt, y_cnt, x_cnt = fusion.get_cell_count_zyx(
        output_volume_size, CELL_SIZE
    )
    for z in range(z_cnt):
        for y in range(y_cnt):
            for x in range(x_cnt):
                worker_cells.append((z, y, x))
    RUNTIME_PARAMS.worker_cells = worker_cells    

    fusion.run_fusion(
            DATASET,
            OUTPUT_PARAMS,
            RUNTIME_PARAMS,
            CELL_SIZE,
            POST_REG_TFMS,
            BLENDING_MODULE,
    )

    # Log 'done' file for next capsule in pipeline. 
    # Unique log filename
    unique_id = str(uuid.uuid4())
    timestamp = int(time.time() * 1000)
    unique_file_name = str(Path(output_path) / f"file_{timestamp}_{unique_id}.yml")

    log_content = {}
    log_content['output_path'] = OUTPUT_PARAMS.path
    log_content['resolution_zyx'] = list(OUTPUT_PARAMS.resolution_zyx)

    with open(unique_file_name, "w") as file:
        yaml.dump(log_content, file)


def execute_job(yml_path: str, 
                xml_path: str,
                output_path: str):
    """
    yml_path: Local yml path
    xml_path: Local xml path 
    output_path: Local results path
    """

    # Parse Pipeline Type:
    configs = read_config_yaml(yml_path)
    pipeline = configs['pipeline']

    # EXASPIM WORKER
    if pipeline == 'exaspim': 
        run_exaspim_worker(yml_path,
                           xml_path, 
                           output_path)

    # DISPIM WORKER
    elif pipeline == 'dispim':
        run_dispim_worker(yml_path, 
                          xml_path, 
                          output_path)

if __name__ == '__main__':
    directory_to_search = "/data/"
    yml_files = [os.path.join(root, file)
                for root, dirs, files in os.walk(directory_to_search)
                for file in files
                if file.endswith(".yml") and not file.startswith('s3')]
    yml_path = str(yml_files[0])
    
    xml_paths = glob.glob('/data/**/*.xml')
    if len(xml_paths) == 0:
        directory_to_search = "../data/"
        xml_paths = [os.path.join(root, file)
                    for root, dirs, files in os.walk(directory_to_search)
                    for file in files
                    if file.endswith((".xml"))]
    xml_path = str(xml_paths[0])

    output_path = str(os.path.abspath('../results'))

    print(f'{yml_path=}')
    print(f'{xml_path=}')
    print(f'{output_path=}')

    execute_job(yml_path, 
                xml_path, 
                output_path)

yml_path='/data/config_561.yml'
xml_path='/data/Christian-ME-New-Lasers_stitch_output/bigstitcher_Christian-ME-New-Lasers.xml'
output_path='/root/capsule/code/aind-cloud-fusion/src/results'
Loading Tile 0 / 4
Loading Tile 1 / 4
Loading Tile 2 / 4
Loading Tile 3 / 4
Loading Tile 0 / 4
Loading Tile 1 / 4
Loading Tile 2 / 4
Loading Tile 3 / 4


2024-01-18 00:42 Number of Tiles: 4
2024-01-18 00:42 output_volume_size=(5399, 4540, 1429)
2024-01-18 00:42 Coloring 18576
2024-01-18 00:42 Calculating 4999/18576...
